In [6]:
import qiskit
from qiskit.circuit.library import QFT
from circuit_knitting.cutting.cutqc import cut_circuit_wires
import mitiq
from qiskit.qasm2 import dumps
from qiskit.providers.fake_provider.backends_v1 import Fake20QV1
from circuit_knitting.cutting.cutqc import reconstruct_full_distribution, evaluate_subcircuits


In [12]:
qf = QFT(4)
qf = qf.decompose()
qf.draw()


┌───┐»
q_0: ──────────────────────────────────────■─────────────■────────■───────┤ H ├»
                                           │       ┌───┐ │        │P(π/2) └───┘»
q_1: ────────────────────■────────■────────┼───────┤ H ├─┼────────■─────────X──»
                   ┌───┐ │        │P(π/2)  │       └───┘ │P(π/4)            │  »
q_2: ──────■───────┤ H ├─┼────────■────────┼─────────────■──────────────────X──»
     ┌───┐ │P(π/2) └───┘ │P(π/4)           │P(π/8)                             »
q_3: ┤ H ├─■─────────────■─────────────────■───────────────────────────────────»
     └───┘                                                                     »
«        
«q_0: ─X─
«      │ 
«q_1: ─┼─
«      │ 
«q_2: ─┼─
«      │ 
«q_3: ─X─
«

In [3]:
cuts = cut_circuit_wires(
    circuit=qf,
    method="automatic",
    max_subcircuit_width=5,
    max_cuts=2,
    num_subcircuits=[2],
)



Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 37 rows and 8 columns.
MIP Presolve modified 7 coefficients.
Aggregator did 65 substitutions.
Reduced MIP has 206 rows, 73 columns, and 600 nonzeros.
Reduced MIP has 67 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.29 ticks)
Found incumbent of value 2.000000 after 0.01 sec. (1.39 ticks)
Probing fixed 24 vars, tightened 3 bounds.
Probing changed sense of 32 constraints.
Probing time = 0.00 sec. (0.82 ticks)
Cover probing fixed 4 vars, tightened 18 bounds.
Tried aggregator 2 times.
MIP Presolve eliminated 185 rows and 49 columns.
MIP Presolve modified 13 coefficients.
Aggregator did 13 substitutions.
Reduced MIP has 8 rows, 11 columns, and 35 nonzeros.
Reduced M

In [5]:
from circuit_knitting.cutting.cutqc import evaluate_subcircuits

subcircuit_instance_probabilities = evaluate_subcircuits(cuts)
print(subcircuit_instance_probabilities[0])

{0: array([1.]), 12: array([0.]), 3: array([0.]), 15: array([0.]), 1: array([1.]), 13: array([0.]), 2: array([0.]), 14: array([0.]), 4: array([1.]), 7: array([0.]), 5: array([1.]), 6: array([0.]), 8: array([0.]), 11: array([0.]), 9: array([0.]), 10: array([0.])}


In [10]:
from cirq.contrib.qasm_import import circuit_from_qasm
qf = qiskit.QuantumCircuit(4)
qf.h(0)
qf.cx(0,1)
qf.cx(1,2)
qf.cx(2,3)
qf.measure_all()
def is_circuit_clifford(circuit, skip_transpile = False):
    if not skip_transpile:
        rb = qiskit.transpiler.passes.RemoveBarriers()
        circuit = qiskit.transpile(circuit, backend=Fake20QV1())
        
        circuit = rb(circuit)
    cqasm = dumps(circuit)
    circuit = circuit_from_qasm(cqasm)
    return mitiq.cdr.clifford_utils.is_clifford(circuit)

is_circuit_clifford(qf)

True

In [13]:
def SplitandCombine(circuit):
    cuts = cut_circuit_wires(
    circuit=circuit,
    method="automatic",
    max_subcircuit_width=5,
    max_cuts=2,
    num_subcircuits=[2])
    eval_list = []
    print(cuts)
    for c in cuts['subcircuits']:
        if c.num_qubits < 2:
            eval_list.append('aer_simulator')
        if is_circuit_clifford(c):
            eval_list.append('aer_simulator')
        else:
            eval_list.append('ibmq_qasm_simulator')
    
    subcircuit_instance_probabilities = evaluate_subcircuits(cuts, backend_names=eval_list)
    reconstructed_probabilities = reconstruct_full_distribution(circuit, subcircuit_instance_probabilities, cuts)

    return reconstructed_probabilities

SplitandCombine(qf)

Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 37 rows and 8 columns.
MIP Presolve modified 7 coefficients.
Aggregator did 65 substitutions.
Reduced MIP has 206 rows, 73 columns, and 600 nonzeros.
Reduced MIP has 67 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.29 ticks)
Found incumbent of value 2.000000 after 0.01 sec. (1.39 ticks)
Probing fixed 24 vars, tightened 3 bounds.
Probing changed sense of 32 constraints.
Probing time = 0.00 sec. (0.82 ticks)
Cover probing fixed 4 vars, tightened 18 bounds.
Tried aggregator 2 times.
MIP Presolve eliminated 185 rows and 49 columns.
MIP Presolve modified 13 coefficients.
Aggregator did 13 substitutions.
Reduced MIP has 8 rows, 11 columns, and 35 nonzeros.
Reduced M

{'max_subcircuit_width': 5, 'subcircuits': [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3dde7687c0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3dde768850>], 'complete_path_map': {Qubit(QuantumRegister(4, 'q'), 0): [{'subcircuit_idx': 1, 'subcircuit_qubit': Qubit(QuantumRegister(4, 'q'), 0)}], Qubit(QuantumRegister(4, 'q'), 1): [{'subcircuit_idx': 1, 'subcircuit_qubit': Qubit(QuantumRegister(4, 'q'), 1)}], Qubit(QuantumRegister(4, 'q'), 2): [{'subcircuit_idx': 0, 'subcircuit_qubit': Qubit(QuantumRegister(2, 'q'), 0)}, {'subcircuit_idx': 1, 'subcircuit_qubit': Qubit(QuantumRegister(4, 'q'), 2)}], Qubit(QuantumRegister(4, 'q'), 3): [{'subcircuit_idx': 0, 'subcircuit_qubit': Qubit(QuantumRegister(2, 'q'), 1)}, {'subcircuit_idx': 1, 'subcircuit_qubit': Qubit(QuantumRegister(4, 'q'), 3)}]}, 'num_cuts': 2, 'counter': {0: {'effective': 0, 'rho': 0, 'O': 2, 'd': 2, 'depth': 3, 'size': 3}, 1: {'effective': 4, 'rho': 2, 'O': 0, 'd': 4, 'depth': 6, 'size': 9}}, 'c

array([0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625])